In [25]:
import numpy as np
import pandas as pd

health = pd.read_csv("../Datasets/rankmd.csv", delimiter=";")
FastFood = pd.read_csv("../Datasets/FastFoodRestaurants.csv")
income = pd.read_csv("../Datasets/kaggle_income.csv", encoding="ISO 8859-1")

us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}
    
# Inverting the dictionary
abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))

# Creating a state dataset
FastFood['State'] = FastFood['province'].map(abbrev_to_us_state)
States = health.copy()
States = States[States['FIPS'].astype(str).str.endswith('000')]

# Converting FIPS to string
health['FIPS']=health['FIPS'].apply(lambda x: '{0:0>5}'.format(x))
health["food_environment_index_Food Environment Index"] = health["food_environment_index_Food Environment Index"].str.replace(",",".").astype(float)

In [26]:
# Removing ' County' from the county names in income
income["County"] = income.County.str.replace(' County', '')

# Merging income and health data
temp_df = income.groupby(["State_Name","County"]).mean().reset_index()
new_df = pd.merge(health.copy(), temp_df.copy(),  how='left', left_on=['State','County'], right_on = ['State_Name','County'])

In [27]:
#ONLY NEEDS TO BE RAN ONCE AS THE COUNTIES ARE STORED IN THE CSV.

#!{sys.executable} -m pip install pgeocode
#import pgeocode

#nomi = pgeocode.Nominatim('us')
#county_names = []
#for i in range(len(FastFood)):
#    county_names.append(nomi.query_postal_code(FastFood["postalCode"][i]).county_name)
    
#FastFood["County"] = county_names
#FastFood.to_csv("../Datasets/FastFoodRestaurants.csv")

In [28]:
# Merging fastfood data with income and health data
temptemp = FastFood.groupby(["State", "County"]).count().reset_index()[['State','County','address']]
tempo = temptemp.rename(columns={'address':'nr of FFchains'})
data_df = pd.merge(new_df, tempo,  how='left', left_on=['State','County'], right_on =['State','County'])
data_df['nr of FFchains'] = data_df['nr of FFchains'].fillna(0)

# Snipping the columns to a more clean dataset
data = data_df[["FIPS", "State", "County", "premature_deathYears_of_Potential_Life_Lost_Rate",'adult_obesity_% Adults with Obesity',
                 "adult_smoking_% Smokers", "excessive_drinking_% Excessive Drinking", "food_environment_index_Food Environment Index",
                 "uninsured_% Uninsured", "unemployed_% Unemployed", "poor_physical_health_days_Average Number of Physically Unhealthy Days",
                "poor_mental_health_days_Average Number of Mentally Unhealthy Days", 'nr of FFchains', 'Mean']]
#data = data.dropna()

data['is_obese'] = data['adult_obesity_% Adults with Obesity']>=33
#data = data.drop(['adult_obesity_% Adults with Obesity'],axis=1)
#data["unemployed_% Unemployed"] = data["unemployed_% Unemployed"].str.replace(",",".").astype(float).astype(int)

/var/folders/p7/1qllz46s60lfswf2l4vv4qtr0000gn/T/ipykernel_50116/4221611364.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['is_obese'] = data['adult_obesity_% Adults with Obesity']>=33


In [29]:
# Storing the merged, cleansed dataset as a csv file
data.to_csv("../Datasets/Mixed_data.csv")
data_df.to_csv("../Datasets/All_data.csv")

In [30]:
data

,FIPS,State,County,premature_deathYears_of_Potential_Life_Lost_Rate,adult_smoking_% Smokers,excessive_drinking_% Excessive Drinking,food_environment_index_Food Environment Index,uninsured_% Uninsured,unemployed_% Unemployed,poor_physical_health_days_Average Number of Physically Unhealthy Days,poor_mental_health_days_Average Number of Mentally Unhealthy Days,nr of FFchains,Mean,is_obese
0,01000,Alabama,NaN,9820.0,20,15,5.5,12.0,"3,0","4,4","4,9",0.0,NaN,True
1,01001,Alabama,Autauga,7830.0,20,14,6.7,10.0,"2,7","4,5","4,9",0.0,53735.557235,True
2,01003,Alabama,Baldwin,7680.0,19,19,7.8,13.0,"2,7","3,6","4,8",19.0,NaN,False
3,01005,Alabama,Barbour,11477.0,26,12,5.5,14.0,"3,8","5,6","5,6",1.0,37725.000000,True
4,01007,Alabama,Bibb,12173.0,23,15,7.6,11.0,"3,1","4,9","5,3",0.0,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3188,56037,Wyoming,Sweetwater,7775.0,19,19,7.5,11.0,"3,9","3,6","4,0",2.0,0.000000,False
3189,56039,Wyoming,Teton,2980.0,13,24,8.3,13.0,"2,7","2,9","3,3",4.0,NaN,False
3190,56041,Wyoming,Uinta,8081.0,21,18,7.3,12.0,"3,9","4,0","4,2",2.0,89130.000000,True
3191,56043,Wyoming,Washakie,6541.0,19,19,8.0,15.0,"3,9","3,7","4,1",0.0,NaN,False
